In [1]:
import os
from tqdm import tqdm
import cv2
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from numpy import asarray

In [2]:
longitud, altura = 224, 224
modelo = './modeloResnet50/modelo_tf_resnet-exp1.h5'
pesos_modelo = './modeloResnet50/pesos_tf_resnet-exp1.h5'
cnn = load_model(modelo)
cnn.load_weights(pesos_modelo)

In [3]:
def Dataset_loader(DIR, RESIZE):
  IMG = []
  read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
  for IMAGE_NAME in tqdm(os.listdir(DIR)):
    PATH = os.path.join(DIR, IMAGE_NAME)
    _, ftype = os.path.splitext(PATH)
    if (ftype == ".jpg" or ftype=='.jpeg'):
      img = read(PATH)
      img = cv2.resize(img, (RESIZE, RESIZE))
      IMG.append(np.array(img) / 255.)
  return IMG

In [4]:
def Dataset_loader2(DIR, RESIZE):
  IMG = []
  for IMAGE_NAME in tqdm(os.listdir(DIR)):
    PATH = os.path.join(DIR, IMAGE_NAME)
    _, ftype = os.path.splitext(PATH)
    if (ftype == ".jpg" or ftype=='.jpeg'):
      image = Image.open(PATH).resize((RESIZE, RESIZE))
      data = asarray(image)
      IMG.append(np.array(data) / 255.)
  return IMG

In [7]:
train_benign = np.array(Dataset_loader('./DermMel/train_sep/NotMelanoma', altura))
train_malign = np.array(Dataset_loader('./DermMel/train_sep/Melanoma', altura))


benign_train_label = np.zeros(len(train_benign))
malign_train_label = np.ones(len(train_malign))

X_train = np.concatenate((train_benign, train_malign), axis = 0)
Y_train = np.concatenate((benign_train_label, malign_train_label), axis = 0)

100%|██████████| 5341/5341 [00:24<00:00, 216.41it/s]


In [8]:
y_pred = cnn.predict(X_train)
print("Done.")

Done.


In [9]:
#Segundo conjunto de imágenes
train_benign2 = np.array(Dataset_loader('./input/skin-cancer-malignant-vs-benign/train/benign', altura))
train_malign2 = np.array(Dataset_loader('./input/skin-cancer-malignant-vs-benign/train/malignant', altura))


benign_train_label2 = np.zeros(len(train_benign2))
malign_train_label2 = np.ones(len(train_malign2))

X_train2 = np.concatenate((train_benign2, train_malign2), axis = 0)
Y_train2 = np.concatenate((benign_train_label2, malign_train_label2), axis = 0)

100%|██████████| 1197/1197 [00:05<00:00, 210.52it/s]


In [10]:
y_pred4 = cnn.predict(X_train2)
print("Done.")

Done.


In [11]:
def ValidationTraining(y_pred,Y_test):
  print(len(y_pred))
  for j in range(50,0,-2):
      predictions = []
      truePositives = 0
      falsePositives = 0
      trueNegatives = 0
      falseNegatives = 0
      for i in range(len(y_pred)):
        if y_pred[i][0] > j/100:
          predictions.append(1)
          if Y_test[i]==1:
              truePositives+=1
          else: falsePositives+=1
        else:
          predictions.append(0)
          if Y_test[i]==0:
              trueNegatives+=1
          else: falseNegatives+=1

      sensitivity=truePositives/(truePositives+falseNegatives)
      especificity=trueNegatives/(trueNegatives+falsePositives)
      accuracy=accuracy_score(Y_test, predictions)
      print("ClassificationNumber:",j/100)
      print("Accuracy model: ",accuracy)
      print("Sensitivity: ",sensitivity)
      print("Especificity: ",especificity)
      print("TP",truePositives,"FN",falseNegatives,"TN",trueNegatives,"FP",falsePositives)
      print("-----------------------------------------------------------------")

In [12]:
ValidationTraining(y_pred,Y_train)

10682
ClassificationNumber: 0.5
Accuracy model:  0.7507021157086687
Sensitivity:  0.6869500093615428
Especificity:  0.8144542220557948
TP 3669 FN 1672 TN 4350 FP 991
-----------------------------------------------------------------
ClassificationNumber: 0.48
Accuracy model:  0.7618423516195469
Sensitivity:  0.7419958809211759
Especificity:  0.781688822317918
TP 3963 FN 1378 TN 4175 FP 1166
-----------------------------------------------------------------
ClassificationNumber: 0.46
Accuracy model:  0.7720464332522
Sensitivity:  0.7979779067590339
Especificity:  0.746114959745366
TP 4262 FN 1079 TN 3985 FP 1356
-----------------------------------------------------------------
ClassificationNumber: 0.44
Accuracy model:  0.7799101291892904
Sensitivity:  0.8498408537727017
Especificity:  0.709979404605879
TP 4539 FN 802 TN 3792 FP 1549
-----------------------------------------------------------------
ClassificationNumber: 0.42
Accuracy model:  0.7761655120763902
Sensitivity:  0.890095487736

In [13]:
ValidationTraining(y_pred4,Y_train2)

2637
ClassificationNumber: 0.5
Accuracy model:  0.6863860447478195
Sensitivity:  0.45363408521303256
Especificity:  0.8798611111111111
TP 543 FN 654 TN 1267 FP 173
-----------------------------------------------------------------
ClassificationNumber: 0.48
Accuracy model:  0.7064846416382252
Sensitivity:  0.5213032581453634
Especificity:  0.8604166666666667
TP 624 FN 573 TN 1239 FP 201
-----------------------------------------------------------------
ClassificationNumber: 0.46
Accuracy model:  0.7136897990140311
Sensitivity:  0.5739348370927319
Especificity:  0.8298611111111112
TP 687 FN 510 TN 1195 FP 245
-----------------------------------------------------------------
ClassificationNumber: 0.44
Accuracy model:  0.7189988623435722
Sensitivity:  0.6282372598162071
Especificity:  0.7944444444444444
TP 752 FN 445 TN 1144 FP 296
-----------------------------------------------------------------
ClassificationNumber: 0.42
Accuracy model:  0.7212741751990899
Sensitivity:  0.6741854636591479

In [14]:
#TESTING

In [10]:
#Para este modelo se ha encontrado que el mejor resultado de corte es 0.08
def ValidationTesting(y_pred,Y_test):
  classificationNumber=0.4
  predictions = []
  truePositives = 0
  falsePositives = 0
  trueNegatives = 0
  falseNegatives = 0
  for i in range(len(y_pred)):
    if y_pred[i][0] > classificationNumber:
      predictions.append(1)
      if Y_test[i]==1:
          truePositives+=1
      else: falsePositives+=1
    else:
      predictions.append(0)
      if Y_test[i]==0:
          trueNegatives+=1
      else: falseNegatives+=1

  sensitivity=truePositives/(truePositives+falseNegatives)
  especificity=trueNegatives/(trueNegatives+falsePositives)
  accuracy=accuracy_score(Y_test, predictions)
  print("ClassificationNumber:",classificationNumber)
  print("Accuracy model: ",accuracy)
  print("Sensitivity: ",sensitivity)
  print("Especificity: ",especificity)
  print("TP",truePositives,"FN",falseNegatives,"TN",trueNegatives,"FP",falsePositives)
  print("-----------------------------------------------------------------")

In [5]:
import random
random.seed(30)

test_benign = np.array(Dataset_loader('./DermMel/test/NotMelanoma', altura))
test_malign = np.array(Dataset_loader('./DermMel/test/Melanoma', altura))


benign_test_label = np.zeros(len(test_benign))
malign_test_label = np.ones(len(test_malign))

X_test = np.concatenate((test_benign, test_malign), axis = 0)
Y_test = np.concatenate((benign_test_label, malign_test_label), axis = 0)

random.shuffle(X_test)
random.shuffle(Y_test)

X_experimentsArray=np.array_split(X_test, 13)
Y_experimentsArray=np.array_split(Y_test, 13)

X_13E=[]
Y_13E=[]

print(len(test_benign),len(test_malign))
rangeNumber=0
for i in range(0,13):
    #100 imagenes benignas
    benign=(test_benign[rangeNumber:rangeNumber+100])
    zeros=(np.zeros(100))
    #100 imagenes malignas
    malign=(test_malign[rangeNumber:rangeNumber+100])
    ones=(np.ones(100))
    
    rangeNumber+=100
    print(rangeNumber)
    X_13E.append(np.concatenate((benign,malign),axis=0))
    Y_13E.append(np.concatenate((zeros,ones),axis=0))  


100%|██████████| 1781/1781 [00:05<00:00, 341.70it/s]


1780 1781
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


In [7]:
#Predicción (solo se ejecuta una vez)
y_pred2 = cnn.predict(X_test)
print("Done.")

Done.


In [8]:
print(len(X_test),len(Y_test),len(malign_test_label),len(benign_test_label),len(test_benign),len(test_malign))

3561 3561 1781 1780 1780 1781


In [9]:
ValidationTesting(y_pred2,Y_test)

ClassificationNumber: 0.4
Accuracy model:  0.7576523448469531
Sensitivity:  0.9101628298708591
Especificity:  0.6050561797752809
TP 1621 FN 160 TN 1077 FP 703
-----------------------------------------------------------------


In [8]:
predictionsArray=[]
for experiment in X_13E:
   predictionsArray.append(cnn.predict(experiment)) 

In [11]:
print(len(X_13E),len(Y_13E),len(predictionsArray))
for i in range(len(predictionsArray)):
    print("Experimento ",i)
    ValidationTesting(predictionsArray[i],Y_13E[i])
    print("---------------------------------------------")

13 13 13
Experimento  0
ClassificationNumber: 0.4
Accuracy model:  0.845
Sensitivity:  0.94
Especificity:  0.75
TP 94 FN 6 TN 75 FP 25
-----------------------------------------------------------------
---------------------------------------------
Experimento  1
ClassificationNumber: 0.4
Accuracy model:  0.75
Sensitivity:  0.91
Especificity:  0.59
TP 91 FN 9 TN 59 FP 41
-----------------------------------------------------------------
---------------------------------------------
Experimento  2
ClassificationNumber: 0.4
Accuracy model:  0.745
Sensitivity:  0.92
Especificity:  0.57
TP 92 FN 8 TN 57 FP 43
-----------------------------------------------------------------
---------------------------------------------
Experimento  3
ClassificationNumber: 0.4
Accuracy model:  0.77
Sensitivity:  0.9
Especificity:  0.64
TP 90 FN 10 TN 64 FP 36
-----------------------------------------------------------------
---------------------------------------------
Experimento  4
ClassificationNumber: 0.4


In [12]:
#Segundo dataset de testing
import random
random.seed(30)

test_benign2 = np.array(Dataset_loader2('./input/skin-cancer-malignant-vs-benign/test/benign', altura))
test_malign2 = np.array(Dataset_loader2('./input/skin-cancer-malignant-vs-benign/test/malignant', altura))

benign_test_label2 = np.zeros(len(test_benign2))
malign_test_label2 = np.ones(len(test_malign2))

X_test2 = np.concatenate((test_benign2, test_malign2), axis = 0)
Y_test2 = np.concatenate((benign_test_label2, malign_test_label2), axis = 0)

X_13E_2=[]
Y_13E_2=[]

print(len(test_malign2),len(test_benign2))
rangeNumber=0
for i in range(0,13):
    #20 imagenes benignas
    benign=(test_benign2[rangeNumber:rangeNumber+20])
    zeros=(np.zeros(20))
    #20 imagenes malignas
    malign=(test_malign2[rangeNumber:rangeNumber+20])
    ones=(np.ones(20))
    
    rangeNumber+=20
    print(rangeNumber)
    X_13E_2.append(np.concatenate((benign,malign),axis=0))
    Y_13E_2.append(np.concatenate((zeros,ones),axis=0))  

100%|██████████| 300/300 [00:00<00:00, 499.70it/s]


300 360
20
40
60
80
100
120
140
160
180
200
220
240
260


In [12]:
y_pred3 = cnn.predict(X_test2)
print("Done.")

Done.


In [13]:
ValidationTesting(y_pred3,Y_test2)

ClassificationNumber: 0.4
Accuracy model:  0.7333333333333333
Sensitivity:  0.75
Especificity:  0.7194444444444444
TP 225 FN 75 TN 259 FP 101
-----------------------------------------------------------------


In [13]:
predictionsArray2=[]
for experiment in X_13E_2:
   predictionsArray2.append(cnn.predict(experiment)) 

In [14]:
print(len(X_13E_2[1]),len(Y_13E_2[1]),len(predictionsArray2[1]))
for i in range(len(predictionsArray2)):
    print("Experimento ",i)
    print(len(predictionsArray2[i]),len(Y_13E_2[i]))
    ValidationTesting(predictionsArray2[i],Y_13E_2[i])
    print("---------------------------------------------")

40 40 40
Experimento  0
40 40
ClassificationNumber: 0.4
Accuracy model:  0.75
Sensitivity:  0.85
Especificity:  0.65
TP 17 FN 3 TN 13 FP 7
-----------------------------------------------------------------
---------------------------------------------
Experimento  1
40 40
ClassificationNumber: 0.4
Accuracy model:  0.75
Sensitivity:  0.8
Especificity:  0.7
TP 16 FN 4 TN 14 FP 6
-----------------------------------------------------------------
---------------------------------------------
Experimento  2
40 40
ClassificationNumber: 0.4
Accuracy model:  0.75
Sensitivity:  0.8
Especificity:  0.7
TP 16 FN 4 TN 14 FP 6
-----------------------------------------------------------------
---------------------------------------------
Experimento  3
40 40
ClassificationNumber: 0.4
Accuracy model:  0.825
Sensitivity:  0.8
Especificity:  0.85
TP 16 FN 4 TN 17 FP 3
-----------------------------------------------------------------
---------------------------------------------
Experimento  4
40 40
Classi